# GeometryNode
The GeometryNode class in Geomapi is the abstract metadata class that governs geometry concepts such as the cartesianBounds and the OrientedBoundingBox. While this node should not be frequently used (unless to govern unknown geospatial data), it houses the geometry function for the MeshNode, BIMNode and PointCloudNode.

**NOTE**: the serialisation of the geometry concepts is mostly based on the [OPENLABEL](https://www.asam.net/index.php?eID=dumpFile&t=f&f=3876&token=413e8c85031ae64cc35cf42d0768627514868b2f#) concept and [Open3D](http://www.open3d.org/docs/release/python_api/open3d.geometry.html) geometry definitions  

The code below shows how to create a GeometryNode class works and how it interacts with RDF Graphs.

First the geomapi and external packages are imported

In [75]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef, Literal
import open3d as o3d
import os
from pathlib import Path
import numpy as np

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

In [76]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
%autoreload 2

## GeometryNode with cartesianBounds

cartesianBounds (np.array [6x1]) [xMin,xMax,yMin,yMax,zMin,zMax], can be set from 5 different inputs:


In [78]:
path = os.path.join(Path(os.getcwd()).parents[2],'test','testfiles','MESH','week22.obj')

mesh=o3d.io.read_triangle_mesh(path)
box=mesh.get_oriented_bounding_box()
box.color=np.array([1,0,0])
alignedBox=mesh.get_axis_aligned_bounding_box()
alignedBox.color=np.array([0,0,1])
vectorlist=box.get_box_points()
pcd=o3d.geometry.PointCloud()
pcd.points=o3d.utility.Vector3dVector(np.vstack((mesh.get_min_bound(),mesh.get_max_bound())))
pcd.paint_uniform_color([0,0,1])
pcd2=o3d.geometry.PointCloud()
pcd2.points=vectorlist
pcd2.paint_uniform_color([1,0,0])


PointCloud with 8 points.

![rendering](../../pics/geometryNode1.PNG)

1. np.array(6x1), list (6 elements) <br>

In [79]:
cartesianBounds=np.array([1,2,3,4,5,6])
node= GeometryNode(cartesianBounds=cartesianBounds)
print(node.cartesianBounds)

[1 2 3 4 5 6]


2. Vector3dVector (n elements)<br>


In [80]:

print(vectorlist)
node= GeometryNode(cartesianBounds=vectorlist)
print(node.cartesianBounds)

std::vector<Eigen::Vector3d> with 8 elements.
Use numpy.asarray() to access data.
[-52.61117755 122.50257926  12.05279306 165.88417048  -0.70982552
  24.03564393]


3. orientedBounds (np.array(8x3))<br>


In [81]:
array=np.asarray(vectorlist)
print(array)
node= GeometryNode(cartesianBounds=array)
print(node.cartesianBounds)

[[-1.96025705e+01  1.65884170e+02  2.22874728e+01]
 [ 1.22465481e+02  1.23859452e+02  2.29468259e+01]
 [-5.26111776e+01  5.43129171e+01  2.33762909e+01]
 [-1.95654721e+01  1.65648765e+02 -7.09825518e-01]
 [ 8.94939722e+01  1.20527931e+01  1.03834556e+00]
 [-5.25740791e+01  5.40775120e+01  3.78992525e-01]
 [ 1.22502579e+02  1.23624046e+02 -5.04724793e-02]
 [ 8.94568738e+01  1.22881982e+01  2.40356439e+01]]
[-52.61117755 122.50257926  12.05279306 165.88417048  -0.70982552
  24.03564393]


4. Open3D.geometry.OrientedBoundingBox<br>


In [82]:
print(box)
node= GeometryNode(cartesianBounds=box)
print(node.cartesianBounds)

OrientedBoundingBox: center: (34.9457, 88.9685, 11.6629), extent: 148.155, 116.357, 22.9985)
[-52.61117755 122.50257926  12.05279306 165.88417048  -0.70982552
  24.03564393]


5. Open3D geometry<br>

In [83]:
print(mesh)
node= GeometryNode(cartesianBounds=mesh)
print(node.cartesianBounds)

TriangleMesh with 330263 points and 485077 triangles.
[-37.36532974 106.94235229  16.87863541 130.69406128   0.71651864
  23.73304558]


### CartesianBounds serialization

cartesianBounds is serialized conform E57 as a string formatted np.array [xMin,xMax,yMin,yMax,zMin,zMax]

In [84]:
node= GeometryNode(cartesianBounds=cartesianBounds)
node.to_graph()
print(node.graph.serialize())


@prefix e57: <http://libe57.org#> .
@prefix v4d: <https://w3id.org/v4d/core#> .

<file:///9aa338d7-1a46-11ed-b3cf-c8f75043ce59> a v4d:GeometryNode ;
    e57:cartesianBounds "[1 2 3 4 5 6]" ;
    v4d:name "9aa338d7-1a46-11ed-b3cf-c8f75043ce59" .




## GeometryNode with orientedBounds

orientedBounds (np.array [8x3]), can be set from 3 different inputs:

1. np.array(6x1), list (6 elements) <br>

In [85]:
orientedBounds=np.asarray(vectorlist)
node= GeometryNode(orientedBounds=orientedBounds)
print(node.orientedBounds)

[[-1.96025705e+01  1.65884170e+02  2.22874728e+01]
 [ 1.22465481e+02  1.23859452e+02  2.29468259e+01]
 [-5.26111776e+01  5.43129171e+01  2.33762909e+01]
 [-1.95654721e+01  1.65648765e+02 -7.09825518e-01]
 [ 8.94939722e+01  1.20527931e+01  1.03834556e+00]
 [-5.25740791e+01  5.40775120e+01  3.78992525e-01]
 [ 1.22502579e+02  1.23624046e+02 -5.04724793e-02]
 [ 8.94568738e+01  1.22881982e+01  2.40356439e+01]]


2. Open3D.geometry.OrientedBoundingBox<br>


In [86]:
print(box)
node= GeometryNode(orientedBounds=box)
print(node.orientedBounds)

OrientedBoundingBox: center: (34.9457, 88.9685, 11.6629), extent: 148.155, 116.357, 22.9985)
[[-1.96025705e+01  1.65884170e+02  2.22874728e+01]
 [ 1.22465481e+02  1.23859452e+02  2.29468259e+01]
 [-5.26111776e+01  5.43129171e+01  2.33762909e+01]
 [-1.95654721e+01  1.65648765e+02 -7.09825518e-01]
 [ 8.94939722e+01  1.20527931e+01  1.03834556e+00]
 [-5.25740791e+01  5.40775120e+01  3.78992525e-01]
 [ 1.22502579e+02  1.23624046e+02 -5.04724793e-02]
 [ 8.94568738e+01  1.22881982e+01  2.40356439e+01]]


2. Open3D geometry<br>

In [87]:
print(mesh)
node= GeometryNode(orientedBounds=mesh)
print(node.orientedBounds)

TriangleMesh with 330263 points and 485077 triangles.
[[-1.96025705e+01  1.65884170e+02  2.22874728e+01]
 [ 1.22465481e+02  1.23859452e+02  2.29468259e+01]
 [-5.26111776e+01  5.43129171e+01  2.33762909e+01]
 [-1.95654721e+01  1.65648765e+02 -7.09825518e-01]
 [ 8.94939722e+01  1.20527931e+01  1.03834556e+00]
 [-5.25740791e+01  5.40775120e+01  3.78992525e-01]
 [ 1.22502579e+02  1.23624046e+02 -5.04724793e-02]
 [ 8.94568738e+01  1.22881982e+01  2.40356439e+01]]


### orientedBounds serialization

orientedBounds is serialized conform V4D as a string formatted np.array (8x3)

In [88]:
node= GeometryNode(orientedBounds=mesh)
node.to_graph()
print(node.graph.serialize())

@prefix v4d: <https://w3id.org/v4d/core#> .

<file:///eb728b67-1a46-11ed-8d02-c8f75043ce59> a v4d:GeometryNode ;
    v4d:name "eb728b67-1a46-11ed-8d02-c8f75043ce59" ;
    v4d:orientedBounds """[[-1.96025705e+01  1.65884170e+02  2.22874728e+01]
 [ 1.22465481e+02  1.23859452e+02  2.29468259e+01]
 [-5.26111776e+01  5.43129171e+01  2.33762909e+01]
 [-1.95654721e+01  1.65648765e+02 -7.09825518e-01]
 [ 8.94939722e+01  1.20527931e+01  1.03834556e+00]
 [-5.25740791e+01  5.40775120e+01  3.78992525e-01]
 [ 1.22502579e+02  1.23624046e+02 -5.04724793e-02]
 [ 8.94568738e+01  1.22881982e+01  2.40356439e+01]]""" .




## GeometryNode with orientedBoundingBox

orientedBoundingBox (Open3D.geometry.OriendtedBoundingBox), can be set from 3 different inputs:

1. Open3D.geometry.OrientedBoundingBox<br>

In [89]:
print(box)
node= GeometryNode(orientedBoundingBox=box)
print(node.orientedBoundingBox)

OrientedBoundingBox: center: (34.9457, 88.9685, 11.6629), extent: 148.155, 116.357, 22.9985)
OrientedBoundingBox: center: (34.9457, 88.9685, 11.6629), extent: 148.155, 116.357, 22.9985)


2. Open3D geometry<br>

In [90]:
print(mesh)
node= GeometryNode(orientedBoundingBox=mesh)
print(node.orientedBoundingBox)

TriangleMesh with 330263 points and 485077 triangles.
OrientedBoundingBox: center: (34.9457, 88.9685, 11.6629), extent: 148.155, 116.357, 22.9985)


3. orientedBounds (np.array(8x3))<br>


In [91]:
array=np.asarray(vectorlist)
print(array)
node= GeometryNode(orientedBoundingBox=array)
print(node.orientedBoundingBox)

[[-1.96025705e+01  1.65884170e+02  2.22874728e+01]
 [ 1.22465481e+02  1.23859452e+02  2.29468259e+01]
 [-5.26111776e+01  5.43129171e+01  2.33762909e+01]
 [-1.95654721e+01  1.65648765e+02 -7.09825518e-01]
 [ 8.94939722e+01  1.20527931e+01  1.03834556e+00]
 [-5.25740791e+01  5.40775120e+01  3.78992525e-01]
 [ 1.22502579e+02  1.23624046e+02 -5.04724793e-02]
 [ 8.94568738e+01  1.22881982e+01  2.40356439e+01]]
OrientedBoundingBox: center: (34.9457, 88.9685, 11.6629), extent: 148.155, 116.357, 22.9985)


**IMPORTANT** orientedBoundingBox is currently not serialized as it can be easily reconstructed from orientedBounds.